In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import math

#Importing data
df = pd.read_csv('1year_hourly_data_2015.csv')
# df.describe()

#Creating train and test set 
train=df[0:8755] #80% of data ( 0.8 * 8755 for training the model) 
test=df[8755:] # remaining 20% to test

#Aggregating the dataset at daily level
df.Timestamp = pd.to_datetime(df.dateformat_hour,format='%Y-%m-%d %H:%M:%S') 
df.index = df.Timestamp 
df = df.resample('H').mean() #daily mean
train.Timestamp = pd.to_datetime(train.dateformat_hour,format='%Y-%m-%d %H:%M:%S') 
train.index = train.Timestamp 
train = train.resample('H').mean() 
test.Timestamp = pd.to_datetime(test.dateformat_hour,format='%Y-%m-%d %H:%M:%S') 
test.index = test.Timestamp 
test = test.resample('H').mean()

y_hat_avg = test.copy()
fit1 = sm.tsa.SARIMAX(train, order=(2, 1, 4),seasonal_order=(0,1,1,24)).fit()
y_hat_avg['SARIMA'] = predictions = fit1.predict(start="2016-1-1 00:00:00", end="2016-1-2 00:00:00", dynamic=True)

predictedValues = list()
for t in predictions:
    predictedValues.append(math.ceil(t))

print(predictedValues)

C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  app.launch_new_instance()
C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check 

0


In [6]:
print(y_hat_avg)

                    dateformat_hour  count     SARIMA
2016-01-01 00:00:00             NaN    NaN   0.418653
2016-01-01 01:00:00             NaN    NaN   0.190128
2016-01-01 02:00:00             NaN    NaN   0.647254
2016-01-01 03:00:00             NaN    NaN   0.834851
2016-01-01 04:00:00             NaN    NaN   0.469592
2016-01-01 05:00:00             NaN    NaN   0.559740
2016-01-01 06:00:00             NaN    NaN   0.396667
2016-01-01 07:00:00             NaN    NaN   0.551315
2016-01-01 08:00:00             NaN    NaN  42.405452
2016-01-01 09:00:00             NaN    NaN  43.573562
2016-01-01 10:00:00             NaN    NaN  42.299121
2016-01-01 11:00:00             NaN    NaN  43.488346
2016-01-01 12:00:00             NaN    NaN  43.691476
2016-01-01 13:00:00             NaN    NaN  42.469065
2016-01-01 14:00:00             NaN    NaN  43.638139
2016-01-01 15:00:00             NaN    NaN  43.344912
2016-01-01 16:00:00             NaN    NaN  42.231227
2016-01-01 17:00:00         

In [8]:
import psycopg2

conn = psycopg2.connect("dbname=merakidb user=postgres password=postgres")
cur = conn.cursor()

cur.execute("DELETE FROM meraki.hourly_visitor_predictions")
for i in range(len(predictions)):
    hourCount = i+1
    visitorCount = math.ceil(predictions[i])
    cur.execute("INSERT INTO meraki.hourly_visitor_predictions (dateformat_hour,count) VALUES (%s, %s);", (hourCount, visitorCount))
    conn.commit()

In [12]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import math

#Importing data
df = pd.read_csv('1year_hourly_data_2015.csv')
# df.describe()

#Creating train and test set 
train=df[0:8755] #80% of data ( 0.8 * 8755 for training the model) 
test=df[8755:] # remaining 20% to test

#Aggregating the dataset at daily level
df.Timestamp = pd.to_datetime(df.dateformat_hour,format='%Y-%m-%d %H:%M:%S') 
df.index = df.Timestamp 
df = df.resample('D').sum() #daily mean
train.Timestamp = pd.to_datetime(train.dateformat_hour,format='%Y-%m-%d %H:%M:%S') 
train.index = train.Timestamp 
train = train.resample('D').sum() 
test.Timestamp = pd.to_datetime(test.dateformat_hour,format='%Y-%m-%d %H:%M:%S') 
test.index = test.Timestamp 
test = test.resample('D').sum()

y_hat_avg = test.copy()
fit1 = sm.tsa.SARIMAX(train, order=(2, 1, 4),seasonal_order=(0,1,1,7)).fit()
y_hat_avg['SARIMA'] = predictions = fit1.predict(start="2016-1-1", end="2016-1-6", dynamic=True)

predictedValues = list()
for t in predictions:
    predictedValues.append(math.ceil(t))

print(predictedValues)

import psycopg2

conn = psycopg2.connect("dbname=merakidb user=postgres password=postgres")
cur = conn.cursor()

cur.execute("DELETE FROM meraki.daily_visitor_predictions")
for i in range(0, 7):
    dayCount = i+1
    visitorCount = math.ceil(predictions[i])
    cur.execute("INSERT INTO meraki.daily_visitor_predictions (dateformat_day,count) VALUES (%s, %s);", (dayCount, visitorCount))
    conn.commit()

C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  app.launch_new_instance()
C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check 

[691, 1034, 1028, 703, 707, 705]


IndexError: index out of bounds